<a href="https://colab.research.google.com/github/jimxs74/SC/blob/main/SC61_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SC61 Certification Status**

Tool ini berguna untuk mendapatkan gambaran progress overall SC project secara angka. agar bisa diidentifikasi letak work volume terbanyak ada diSystem/Disiplin mana

UI/UX belum menjadi objective dari tool ini, akan dilanjutkan dengan framework yang lain

In [1]:
import pandas as pd
import numpy as np
import gspread

In [2]:
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()

import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [3]:
def read_data(url, sheet='sheet1'):
  gs = gc.open_by_url(url).worksheet(sheet).get_all_values()
  gs = pd.DataFrame.from_records(gs)
  gs.columns = gs.iloc[0]
  gs = gs[1:]
  return gs

def write_data(url, result, sheet):
  write = gc.open_by_url(url)
  write = write.worksheet(sheet)
  write.clear()
  set_with_dataframe(worksheet=write, dataframe=result, include_index=False,include_column_header=True, resize=True)
  return

Update Data System Owner

In [4]:
# Connect dan Ambil data dari SC18
url_sc18 = 'https://docs.google.com/spreadsheets/d/1wrvBoSNka1f51hk8QuLd_bF7irVzSB6nDpg-Z_XqcG4/edit#gid=532748829'
url_sc61 = 'https://docs.google.com/spreadsheets/d/1vrdgQtQEfgirlOOzfVxcxuZizaETIxQ9XvdN1Qryg40/edit#gid=689240321'

gs_read = gc.open_by_url(url_sc18).worksheet('System List')
gs_write = gc.open_by_url(url_sc61)

df = gs_read.get_all_values()
df = pd.DataFrame.from_records(df)

# Ambil kolom yg di perlukan saja dari SC18
df = df[[1,2,3,4,32]] 
df.columns = ['system','system desc','subsystem','subsystem desc', 'system owner']
df['system'].replace('',np.nan, inplace=True)
df['subsystem'].replace('',np.nan, inplace=True)

# pilih row yang diperlukan saja
df_so = df[6:]
df_so = df_so.drop(['subsystem','subsystem desc'], axis=1)

# drop row yg mengandung blank pada column subsystem
df_so.dropna(subset=['system'], inplace=True)
df_so.reset_index(drop=True)

# save to gsheet diworksheet so
write_data(url_sc61, df_so, 'so')

# Generate summary table Sub-System Owner dari SC18
# pilih row yang diperlukan saja
df_sso = df[6:]
df_sso = df_sso.drop(['system','system desc'], axis=1)

# drop row yg mengandung blank pada column subsystem
df_sso.dropna(subset=['subsystem'], inplace=True)
df_sso.reset_index(drop=True)

# save to gsheet diworksheet sso
write_data(url_sc61, df_sso, 'sso')


#1. Connect ke Data

In [5]:
# link data dari p Supri. di update harian oleh p Supri
# convert to DataFrame
url_cert = "https://docs.google.com/spreadsheets/d/1xPQanRi1XDmo7FlUyiV6Rt8pro_zjEHLOE0GLROTReQ/edit#gid=1035965456"
url_itr = "https://docs.google.com/spreadsheets/d/1aB57SwMedoMSjBxt5w_EBNYgmMoUACTJC056gEbD9a0/edit#gid=1491426671"
url_cc = "https://docs.google.com/spreadsheets/d/1ZrQjq30E58bFFiuuZ9aqAYy2StB4uKDRb2oDtM3m8wk/edit#gid=1688767959"
url_pc = "https://docs.google.com/spreadsheets/d/1DHwpGa_I0sd42o9zfC5cUMSud9XfKwPtyYm7UHY9xl0/edit#gid=1494113475"
url_pl = "https://docs.google.com/spreadsheets/d/19VIlyxleK14LjZCPls-4F9fjg0Sh46Q8oTk5UawJzhI/edit#gid=622809212"
url_sr = "https://docs.google.com/spreadsheets/d/1X3Zy083zX2spgZWqlw9TfnzUpHh_LR_C3H1jXyEdB4A/edit#gid=253348165"

#SC61 Actual
url_sc61 = 'https://docs.google.com/spreadsheets/d/1vrdgQtQEfgirlOOzfVxcxuZizaETIxQ9XvdN1Qryg40/edit#gid=0' # yg asli

# SC61 test
#url_sc61 = 'https://docs.google.com/spreadsheets/d/1LqkgoSVrjKQjVUi1VctEA9SdPBypNEai1HvNjkqQg5w/edit#gid=1588063696' # uji coba

cert = read_data(url_cert,'CERT')
itr = read_data(url_itr,'ITR')
cc = read_data(url_cc,'CC')
pc = read_data(url_pc,'PC')
pl = read_data(url_pl,'PL')
sr = read_data(url_sr,'PHASE')

#2. Certificate connect dengan subsystem owner (cert+sso)

 kelebihan : join system owner lebih specifik pada subsystem, lebih akurat <br>
 kekurangan : ada bug, SHOC/SCCC/RFSU tidak terambil datanya dan teralamatkan ke PIC SO

In [6]:
#ambil data certificate
cert = cert[['Certificate No','System No','System Description','Sub System No','Sub System Description','Discipline','Status']]
#cert['System No'] = pd.to_numeric(cert['System No'], errors='coerce').astype('Int64')

#ambil data SC61 worksheet sso
sso = read_data(url_sc61,'sso')

# Merge data dengan CSystem Report Certificate Status dengan SC61
sert_sso = pd.merge(cert, sso, left_on ="Sub System No", right_on="subsystem" , how="left") # previous is inner
sert_sso = sert_sso.drop(['System No','System Description','subsystem','subsystem desc'], axis=1)

# Sebaran Outstanding Certificate
sert_sso = sert_sso[sert_sso['Status'].str.contains('Outstanding')]
sso_load= sert_sso["system owner"].value_counts()
#print("Beban System Owner dalam subsystem lebih akurat : ")
#print(sso_load)

In [7]:
# save to gsheet diworksheet sso
#write_data(url_sc61, sert_sso, sheet='sso_wf_sert')

In [8]:
#update ke data system owner sertificate, untuk fungsi vlookup di lembar koordinasi
so = read_data(url_sc61,'so')
sert_so = pd.merge(cert, so, left_on ="System No", right_on="system" , how="left") # previosly inner
sert_so = sert_so.drop(['Sub System No','Sub System Description','system','system desc'], axis=1)
sert_so = sert_so.sort_values(by=['System No'])

write_data(url_sc61, sert_so, sheet='so_wf_sert')

#3. Connect to Summary Report (cert+sso+summary report)

In [9]:
sr = sr[['System No', 'System Description', 'Sub System No','Sub System Description', 'Discipline', 'ITR Outstanding', 'CC Checksheet Outstanding', 'PC Checksheet Outstanding', 'COMM Checksheet Outstanding']]
#sr = sr.fillna(0)

sr['System No'] = pd.to_numeric(sr['System No'], errors='coerce').astype('Int64')
sr['ITR Outstanding'] = pd.to_numeric(sr['ITR Outstanding'], errors='coerce').astype('Int64')
sr['CC Checksheet Outstanding'] = pd.to_numeric(sr['CC Checksheet Outstanding'], errors='coerce').astype('Int64')
sr['PC Checksheet Outstanding'] = pd.to_numeric(sr['PC Checksheet Outstanding'], errors='coerce').astype('Int64')
sr['COMM Checksheet Outstanding'] = pd.to_numeric(sr['COMM Checksheet Outstanding'], errors='coerce').astype('Int64')

sr = sr.fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [10]:
# Filter yg masih outstanding saja di bagian CC Checksheet
sr_cc = sr[['Sub System No','Discipline','CC Checksheet Outstanding']]
sr_cc = sr_cc[sr_cc['CC Checksheet Outstanding'] != 0]

# combine text on different column
sr_cc["Discipline"] = sr_cc["Discipline"].astype(str) + ":" + sr_cc["CC Checksheet Outstanding"].astype(str)
sr_cc = sr_cc.drop(columns=['CC Checksheet Outstanding'])
sr_cc = sr_cc.set_index('Sub System No')

# combine value as string ref : https://stackoverflow.com/questions/59914167/putting-all-the-same-value-indices-into-one-row
sr_cc = sr_cc.assign(labels = sr_cc.groupby(level = 'Sub System No').cumcount()).groupby([sr_cc.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)
sr_cc.columns = ['CC Outstanding', 'A','B','C']

sr_cc["CC Outstanding"] = sr_cc["CC Outstanding"].astype(str) + "/" + sr_cc["A"].astype(str) + "/" + sr_cc["B"].astype(str) + "/" + sr_cc["C"].astype(str)
sr_cc = sr_cc.drop(columns=['A','B','C'])

In [11]:
# Filter yg masih outstanding saja di bagian PC Checksheet
sr_pc = sr[['Sub System No','Discipline','PC Checksheet Outstanding']]
sr_pc = sr_pc[sr_pc['PC Checksheet Outstanding'] != 0]

# combine text on different column
sr_pc["Discipline"] = sr_pc["Discipline"].astype(str) + ":" + sr_pc["PC Checksheet Outstanding"].astype(str)
sr_pc = sr_pc.drop(columns=['PC Checksheet Outstanding'])
sr_pc = sr_pc.set_index('Sub System No')

# combine value as string 
sr_pc = sr_pc.assign(labels = sr_pc.groupby(level = 'Sub System No').cumcount()).groupby([sr_pc.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)
sr_pc.columns = ['PC Outstanding', 'A','B','C','D']

sr_pc["PC Outstanding"] = sr_pc["PC Outstanding"].astype(str) + "/" + sr_pc["A"].astype(str) + "/" + sr_pc["B"].astype(str) + "/" + sr_pc["C"].astype(str) + "/" + sr_pc["D"].astype(str)
sr_pc = sr_pc.drop(columns=['A','B','C','D'])

##4. Merge Certificate + SSO + Summary Report

In [12]:
# merge cc_pc
cc_pc = pd.merge(sr_cc, sr_pc, left_on ="Sub System No", right_on="Sub System No" , how="outer")
# perlu ada fungsi untuk cleaning /nan/
cc_pc['CC Outstanding'] = cc_pc['CC Outstanding'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)
cc_pc['PC Outstanding'] = cc_pc['PC Outstanding'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)

In [13]:
# Merge dengan data system owner sert_so dengan sr_pc
sert_sso = pd.merge(sso, cc_pc, left_on ="subsystem", right_on="Sub System No" , how="inner")

#5. Connect to PunchList Data (cert+sso+summary report + punch)

In [14]:
# select only useful column
pl = pl[['Sub System No','Discipline','Category','Punchlist No','Punchlist Description','Status','Phase','Raised By']] 
pl = pl.loc[pl['Status']=='Outstanding'].drop(columns=['Status'])
pl = pl.loc[pl['Category'].isin(['A','B'])]   # saat include C, line ini akan di escape

#membuat buffer percabangan untuk di buat table dedicated dalam worksheet pl_A dan pl_B
pl_a = pl.loc[pl['Category'].isin(['A'])]
pl_b = pl.loc[pl['Category'].isin(['B'])]

In [15]:
# Menghitung jumlah punch saja
pl = pl.groupby(['Sub System No','Discipline','Category'])['Punchlist No'].count().reset_index(name="jumlah")

# Combine text Discipline/Category/Jumlah C:(B=4)
pl["Discipline"] = pl["Discipline"].astype(str) + ":(" + pl["Category"].astype(str) + "=" + pl["jumlah"].astype(str) + ")"

pl = pl.drop(columns=['Category','jumlah'])
pl = pl.set_index('Sub System No')

# combine value as string ref
pl = pl.assign(labels = pl.groupby(level = 'Sub System No').cumcount()).groupby([pl.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)
pl.columns = ['Punch', 'A','B','C','D','E','F'] # jika ada error jumlah element
#pl.columns = ['Punch', 'A','B','C','D','E','F','G']

pl["Punch"] = pl["Punch"].astype(str) + "/" + pl["A"].astype(str) + "/" + pl["B"].astype(str) + "/" + pl["C"].astype(str) + "/" + pl["D"].astype(str) + "/" + pl["E"].astype(str) + "/" + pl["F"].astype(str)
#pl["Punch"] = pl["Punch"].astype(str) + "/" + pl["A"].astype(str) + "/" + pl["B"].astype(str) + "/" + pl["C"].astype(str) + "/" + pl["D"].astype(str) + "/" + pl["E"].astype(str) + "/" + pl["F"].astype(str) + "/" + pl["G"].astype(str)
pl = pl.drop(columns=['A','B','C','D','E','F'])# jika ada error jumlah element
#pl = pl.drop(columns=['A','B','C','D','E','F','G'])

pl['Punch'] = pl['Punch'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)


In [16]:
# Merge dengan data subsystem owner sertificate sert_sso
sert_sso = pd.merge(sert_sso, pl, left_on ="subsystem", right_on="Sub System No" , how="left")

In [17]:
# save to gsheet diworksheet sso_wf_sert
#write_data(url_sc61, sert_sso, 'sso_wf_sert')

#6. Connect to System Priority untuk dimasukan dalam tol Gasin  (cert+sso+summary report + punch + tol)

In [18]:
# Mengambil data dari SC61 worksheet tol gasin dan clean up
tol = read_data(url_sc61,'tol gasin')

# select only yg tol yes
tol = tol.loc[tol['Tol']=='yes']

# merge dengan table di sert_sso (worksheet sso_wf_sert)
sert_ssox = pd.merge(sert_sso, tol, left_on ="subsystem", right_on="subsystem" , how="left")
# reconfigure position column
sert_ssox = sert_ssox.reindex(columns=['subsystem','subsystem desc_x', 'Tol','system owner','CC Outstanding','PC Outstanding','Punch']) # letak posisi salah.

In [19]:
# save to gsheet diworksheet sso_wf_sert
#write_data(url_sc61, sert_ssox, 'sso_wf_sert')

#7. Connect to Data ITR

In [20]:
#Ambil data ITR dan clean up
itr = itr[['Sub System No','Discipline','Tag No','ITR No','ITR Description','Status']]
# ambil hanya yang outstanding
itr = itr.loc[(itr['Status']=='Outstanding')]
# Membuat buffer percabangan untuk membuat tabel worksheet remaining ITR, utk digunakan nanti
itr_show = itr.drop(columns=['Status'])
# Menghitung jumlah ITR per subsystem
itr = itr.groupby(['Sub System No','Discipline'])['ITR No'].count().reset_index(name="jumlah")

# Combine text Discipline/Category/Jumlah C:1
itr["Discipline"] = itr["Discipline"].astype(str) + ":" + itr["jumlah"].astype(str)
itr = itr.drop(columns=['jumlah'])
itr = itr.set_index('Sub System No')

# combine value as string
itr = itr.assign(labels = itr.groupby(level = 'Sub System No').cumcount()).groupby([itr.index,'labels']).first().unstack('labels').sort_index(axis =1,level = 1).droplevel(1,axis = 1)
itr.columns = ['ITR','A','B','C','D','E','F','G','H']

itr["ITR"] = itr["ITR"].astype(str) + "/" + itr["A"].astype(str) + "/" + itr["B"].astype(str) + "/" + itr["C"].astype(str) + "/" + itr["D"].astype(str) + "/" + itr["E"].astype(str) + "/" + itr["F"].astype(str) + "/" + itr["G"].astype(str) + "/" + itr["H"].astype(str)
itr = itr.drop(columns=['A','B','C','D','E','F','G','H'])

# cleaning /nan/
itr['ITR'] = itr['ITR'].str.replace(r'nan[/]','', regex=True).str.replace(r'/nan','', regex=True)

In [21]:
# merge dengan table di sert_sso (worksheet sso_wf_sert)
sert_ssox2 = pd.merge(sert_ssox, itr, left_on ="subsystem", right_on="Sub System No" , how="left")
sert_ssox2 = sert_ssox2.reindex(columns=['subsystem','subsystem desc_x', 'Tol','system owner', 'ITR', 'CC Outstanding','PC Outstanding','Punch'])

In [22]:
# save to gsheet diworksheet sso_wf_sert
write_data(url_sc61, sert_ssox2, 'sso_wf_sert')

#8. Refresh Outstanding ITR/PC/CC/ITR

In [23]:
# merge dengan table tol
itr_show = pd.merge(itr_show, tol, left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur tol
itr_show = itr_show.loc[(itr_show['Tol']=='yes')]
itr_show = itr_show.drop(columns=["subsystem"])

# Update worksheet remaining ITR 
write_data(url_sc61, itr_show, 'itr')

In [24]:
#Ambil data CC dan clean up
cc = cc[['Sub System No','Discipline','Tag No','Checksheet Description','Status']]

# ambil hanya yang outstanding
cc = cc.loc[(cc['Status']=='Outstanding')]
cc = cc.drop(columns=['Status'])

# merge dengan table tol
cc = pd.merge(cc, tol, left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur tol
cc = cc.loc[(cc['Tol']=='yes')]
cc = cc.drop(columns=["subsystem"])

# Update worksheet remaining CC
write_data(url_sc61, cc, 'cc')

In [25]:
# merge dengan table tol
pl_a = pd.merge(pl_a, tol, left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur tol
pl_a = pl_a.loc[(pl_a['Tol']=='yes')]
pl_a = pl_a.drop(columns=["subsystem"])

# Update worksheet remaining PL-A
write_data(url_sc61, pl_a, 'pl_A')

In [26]:
#Ambil data PC dan clean up
pc = pc[['Sub System No','Discipline','Tag No','Checksheet No','Checksheet Description','Status']]

# ambil hanya yang outstanding
pc = pc.loc[(pc['Status']=='Outstanding')]
pc = pc.drop(columns=['Status'])

# merge dengan table tol
pc = pd.merge(pc, tol, left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur tol
pc = pc.loc[(pc['Tol']=='yes')]
pc = pc.drop(columns=["subsystem"])

# Update worksheet remaining PC # di pindahkan ke bawah
#write_data(url_sc61, pc, 'pc')

In [27]:
# merge dengan table tol
pl_b = pd.merge(pl_b, tol, left_on ="Sub System No", right_on="subsystem" , how="left")
# hanya tunjukan yg ada dalam jalur tol
pl_b = pl_b.loc[(pl_b['Tol']=='yes')]
pl_b = pl_b.drop(columns=["subsystem"])

# Update worksheet remaining PL-B
write_data(url_sc61, pl_b, 'pl_B')

#9. Connect to Transmital SC17

In [57]:
# terhubung dengan SC17
url_sc17 = 'https://docs.google.com/spreadsheets/d/1nrdtZOiToC_kW95a8rKR_-J8hH5Io5Rp_JG2a8S2T3o/edit#gid=1274212197'

sc17_cc = read_data(url_sc17,'CS Monitoring ')
sc17_pc = read_data(url_sc17,'PC Monitoring')
sc17_cert = read_data(url_sc17,'Certificate Monitoring')

In [60]:
#ambil data transmital PC yg diperlukan dan clean up
sc17_cc = sc17_cc[['Subsystem','Date','   Discipline','Tag No','Checksheet No','Status','Remarks','Transmital No']]
#merge data
cc_x1 = pd.merge(cc, sc17_cc, left_on ="Tag No", right_on="Tag No" , how="left")

#di atur kolomnya
cc_x1 = cc_x1.reindex(columns=['Sub System No','Discipline', 'Tag No','Checksheet Description', 'subsystem desc','Tol','Date','Status','Remarks','Transmital No'])
cc_x1 = cc_x1.rename(columns={"Date": "SC17_submit Date", "Remarks": "SC17 Status"})

# write cc
write_data(url_sc61, cc_x1, 'cc')

In [50]:
#ambil data transmital PC yg diperlukan dan clean up
sc17_pc = sc17_pc[['Subsystem No.','Date','Discipline','Tag No.','Checksheet Description','Status','Remarks','Transmital No']]

#merge data
pc_x1 = pd.merge(pc, sc17_pc, left_on ="Tag No", right_on="Tag No." , how="left")

#di atur kolomnya
pc_x1 = pc_x1.reindex(columns=['Sub System No','Discipline_x', 'Tag No','Checksheet No', 'Checksheet Description_x',
'Tol','Date','Remarks','Transmital No'])
pc_x1 = pc_x1.rename(columns={"Date": "SC17_submit Date", "Remarks": "SC17 Status"})

# write pc
write_data(url_sc61, pc_x1, 'pc')

In [81]:
sc17_cert = read_data(url_sc17,'Certificate Monitoring')

In [82]:
#ambil data transmital cert yg diperlukan dan clean up
sc17_cert = sc17_cert[['System/sub-system','Date','Discipline','Certificate','Re-submit','Remarks','Transmital No']]

In [83]:
sc17_cert.head(3)

,System/sub-system,Date,Discipline,Certificate,Re-submit,Remarks,Transmital No
1,067-03,20/11/2020,Civil,DCCC&SSCCC,CLOSED,CLOSED 25/11/2020,JTB-T-R-S-0017
2,067-06,20/11/2020,Civil,DCCC&SSCCC,CLOSED,CLOSED 25/11/2020,JTB-T-R-S-0017
3,067-10,20/11/2020,Civil,DCCC&SSCCC,CLOSED,CLOSED 25/11/2020,JTB-T-R-S-0017


In [72]:
sert_ssox2.head()

,subsystem,subsystem desc_x,Tol,system owner,ITR,CC Outstanding,PC Outstanding,Punch
0,068-01,Main Control Building,NaN,Yuniar,C:1,C:1,C:1,NaN
1,068-08,Substation at Sales Gas Metering,NaN,Andre Wisoyo,NaN,NaN,C:1,C:(A=13)/C:(B=1)
2,068-09,Guard House GPF,NaN,Yuniar,C:24,C:4,C:2,E:(A=1)
3,068-10,"Guard House, Security Post Gayam Public Road J...",NaN,Andre Wisoyo,C:14,C:2,C:1,NaN
4,068-11,Reception Building,NaN,Andre Wisoyo,C:31,C:3,C:1,NaN
